In [4]:
# Import your libraries:
%matplotlib inline

import numpy as np
import pandas as pd

salesDF = pd.read_csv('../data/sales.csv')
salesDF

# Challenge 1 - Explorar el Dataset
# Examinar la vista general de los datos
print("Vista general del dataset:")
print(salesDF.describe())

# Examinar los tipos de datos
print("\nTipos de datos:")
print(salesDF.info())

# Verificar si la primera columna tiene valores únicos
primera_columna = salesDF.iloc[:, 0]
es_unica = primera_columna.is_unique

print("¿La primera columna tiene valores únicos?:", es_unica)
print("\nNúmero total de valores:", len(primera_columna))
print("Número de valores únicos:", primera_columna.nunique())

if es_unica:
    # Si son valores únicos, establecemos como índice
    salesDF = salesDF.set_index(salesDF.columns[0])
    salesDF.index.name = 'index'
else:
    print("¡Advertencia! La primera columna contiene valores duplicados y no puede usarse como índice")




Vista general del dataset:
          Unnamed: 0       store_ID    day_of_week  nb_customers_on_day  \
count  640840.000000  640840.000000  640840.000000        640840.000000   
mean   355990.675084     558.211348       4.000189           633.398577   
std    205536.290268     321.878521       1.996478           464.094416   
min         0.000000       1.000000       1.000000             0.000000   
25%    178075.750000     280.000000       2.000000           405.000000   
50%    355948.500000     558.000000       4.000000           609.000000   
75%    533959.250000     837.000000       6.000000           838.000000   
max    712044.000000    1115.000000       7.000000          5458.000000   

                open      promotion  school_holiday          sales  
count  640840.000000  640840.000000   640840.000000  640840.000000  
mean        0.830185       0.381718        0.178472    5777.469011  
std         0.375470       0.485808        0.382910    3851.338083  
min         0.000000 

In [ ]:
# Challenge 2 - Analizar Correlaciones

import seaborn as sns
import matplotlib.pyplot as plt



# Crear matriz de correlación usando columnas numéricas
correlation_matrix = salesDF.select_dtypes(include=['int64', 'float64']).corr()

# Crear heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación de Características')
plt.show()



In [ ]:
# Challenge 3 - Manejar Valores Faltantes

# Examinar valores faltantes
print("Número de valores faltantes por columna:")
print(salesDF.isnull().sum())

# Calcular porcentaje de valores faltantes
print("\nPorcentaje de valores faltantes por columna:")
print((salesDF.isnull().sum() / len(salesDF)) * 100)

# Eliminar columnas con más del 50% de valores faltantes
missing_percentages = (salesDF.isnull().sum() / len(salesDF)) * 100
columns_to_drop = missing_percentages[missing_percentages > 50].index
sales_cleaned = salesDF.drop(columns=columns_to_drop)

# Eliminar filas con valores faltantes
sales_cleaned = sales_cleaned.dropna()




In [ ]:
# Challenge 4 - Preparar Variables Categóricas

# Convertir variables categóricas a dummies
sales_dummy = pd.get_dummies(sales_cleaned, drop_first=True)



In [ ]:
# Challenge 5 - Modelado y Predicción 
# (asumiendo que queremos predecir 'nb_customers_on_day')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Preparar variables
X = sales_dummy.drop('nb_customers_on_day', axis=1)
y = sales_dummy['nb_customers_on_day']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir y evaluar
y_pred = model.predict(X_test)
print("\nResultados del modelo:")
print(f"R2 Score: {r2_score(y_test, y_pred)}")
print(f"MSE: {mean_squared_error(y_test, y_pred)}")




In [ ]:
# Importar las librerías necesarias
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

print("\n=== Análisis de Importancia de Features con XGBoost y RandomForest ===")

# XGBoost
print("\n--- XGBoost ---")
# Definir parámetros de XGBoost para regresión
params_xgb = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 3,
    'random_state': 42
}

# Instanciar y entrenar modelo XGBoost
model_xgb = xgb.XGBRegressor(**params_xgb)
model_xgb.fit(X_train, y_train)

# Hacer predicciones
y_pred_xgb = model_xgb.predict(X_test)

# Evaluar modelo
print("\nResultados XGBoost:")
print(f"R2 Score: {r2_score(y_test, y_pred_xgb):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_xgb):.4f}")

# Visualizar importancia de features con XGBoost
plt.figure(figsize=(12, 6))
sorted_idx = model_xgb.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], model_xgb.feature_importances_[sorted_idx])
plt.title('Importancia de Features - XGBoost')
plt.show()



In [ ]:
# Random Forest
print("\n--- Random Forest ---")
# Definir parámetros de Random Forest
params_rf = {
    'n_estimators': 100,
    'max_depth': 3,
    'random_state': 42
}

# Instanciar y entrenar modelo Random Forest
model_rf = RandomForestRegressor(**params_rf)
model_rf.fit(X_train, y_train)

# Hacer predicciones
y_pred_rf = model_rf.predict(X_test)

# Evaluar modelo
print("\nResultados Random Forest:")
print(f"R2 Score: {r2_score(y_test, y_pred_rf):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_rf):.4f}")

# Visualizar importancia de features con Random Forest
plt.figure(figsize=(12, 6))
sorted_idx = model_rf.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], model_rf.feature_importances_[sorted_idx])
plt.title('Importancia de Features - Random Forest')
plt.show()



In [ ]:
# Comparación de modelos
print("\n=== Comparación de Modelos ===")
modelos = {
    'Regresión Lineal': (model, y_pred),
    'XGBoost': (model_xgb, y_pred_xgb),
    'Random Forest': (model_rf, y_pred_rf)
}

for nombre, (modelo, predicciones) in modelos.items():
    print(f"\n{nombre}:")
    print(f"R2 Score: {r2_score(y_test, predicciones):.4f}")
    print(f"MSE: {mean_squared_error(y_test, predicciones):.4f}")

# Visualización comparativa de predicciones
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title('Regresión Lineal')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')

plt.subplot(132)
plt.scatter(y_test, y_pred_xgb, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title('XGBoost')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')

plt.subplot(133)
plt.scatter(y_test, y_pred_rf, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title('Random Forest')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')

plt.tight_layout()
plt.show()
